# Introduction

This notebook presents **lca-algebraic** a small libray above **brightay2**, designed for the definition of **parametric inventories** with fast computation of LCA impacts, suitable for **monte-carlo** analyis.

**lca-algebraic** provides a set of  **helper functions** for : 
* **compact** & **human readable** definition of activites :  
    * search background (tech and biosphere) activities 
    * create new foreground activites with parametrized amounts
    * parametrize / update existing background activities (extending the class **Activity**)
* Definition of parameters
* Computation of LCAs 
* Computation of statistics (including Sobols indices)
* Compute simplified parametric model by fixing minor input parameters

  
## Principles 

The main idea of this libray is to move from **procedural definition** of models (slow and prone to errors) to a **declarative / purely functionnal** definition of parametric models (models as **pure functions**). 

This enables **fast computation of LCA impacts**, useful for Monte Carlo methods and statistical analysis. 
We leverage the **power of symbolic calculus** provided by the library [SymPy](https://www.sympy.org/en/index.html).

We define our model in a **separate database**, as a nested combination of : 
* other foreground activities
* background activities :
    * Technical, refering **ecoinvent DB**
    * Biosphere, refering **brightway2** biosphere activities
    
The **amounts** in exchanges are expressed either as **static amounts**, or **symbolic expressions** of pre-defined **parameters**.

Each activity of our **root model** is defined as a **parametrized combination** of the **foreground activities**, which can themselves be expressed by the **background activities**.

When computing LCA for foreground models, the library develops the model as a combination of **only background activities**. It computes **once for all** the impact of **all required background activities** and compiles a **fast numpy** (vectorial) function for each impact, replacing each background activity by the **static value of the corresponding impact**.

By providing **large vectors** of **parameter values** to those numpy functions, we can compute LCA for **thousands of values** at a time.

![](https://oie-mines-paristech.github.io/lca_algebraic/doc/lca-algebraic.png)


## Compatiblity with brightway2 

Under the hood, the activities we define with **lca-algebraic** are standard **brightway2** activities. 
The amounts of exchanges are stored as **float values** or **serialized as string** in the property **formula**.

Parameters are also stored in the **brightay2** projets, making it fully compatible with **brightway**.

Thus, a model defined with **lca-algebraic** is stored as a regular **bw2** projet. We can use **bw2** native support for [parametrized dataset](https://2.docs.brightway.dev/intro.html#parameterized-datasets) for computing LCAs, even if much more slower than the method explain here.

## Doc

The followng notebook explores the main functions.
Full documentation of the functions is [available here](https://oie-mines-paristech.github.io/lca_algebraic/doc/)



## Imports / init

In [1]:
# Those two lines are for dev only : they watch imported libraries for changes
%load_ext autoreload
%autoreload 2

import brightway2 as bw
import os 
# Custom utils defined for inter-acv
import lca_algebraic as agb
from lca_algebraic import DistributionType

from sympy import init_printing
import bw2io
from dotenv import load_dotenv

# Pretty print for Sympy
init_printing()

# Init brightway2 and databases

In [3]:
# Set the current project
# Can be any name
bw.projects.set_current('MyLCAAlgebraicProject')

# It's better to not leave credential in the code.
# Create a file named .env, that you will not share /commit, and contains the following :
# ECOINVENT_LOGIN=<your_login>
# ECOINVENT_PASSWORD=<your_password>

# This load .env file into os.environ
load_dotenv()

# XXX: Empty Db
#for db in list(bw.databases):
#    del bw.databases[db]

# This downloads ecoinvent and installas biopshere + technosphere + LCIA methods
if len(bw.databases) > 0:
    print("Initial setup already done, skipping")
else:
    bw2io.import_ecoinvent_release(
        version="3.7",
        system_model="cutoff",
        username=os.environ["ECOINVENT_LOGIN"],
        password=os.environ["ECOINVENT_PASSWORD"],
        use_mp=True)

Vacuuming database 
Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.01 seconds
4318 datasets
0 exchanges
0 unlinked exchanges
  
Title: Writing activities to SQLite3 database:
  Started: 02/22/2024 17:49:19
  Finished: 02/22/2024 17:49:19
  Total time elapsed: 00:00:00
  CPU %: 96.80
  Memory %: 2.83
Created database: ecoinvent-3.7-biosphere
Extracting XML data from 19128 datasets
Extracted 19128 datasets in 46.22 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applyi

In [4]:
# We use a separate DB for defining our foreground model / activities
# Choose any name
USER_DB = 'MyForeground'

# This is better to cleanup the whole foreground model each time, and redefine it in the notebook
# instead of relying on a state or previous run.
# Any persistent state is prone to errors.
agb.resetDb(USER_DB)

# Parameters are stored at project level : 
# Reset them also
# You may remove this line if you import a project and parameters from an external source (see loadParam(..))
agb.resetParams()

# Overview of the databases
agb.list_databases()

,backend,nb_activities,type
name,,,
ecoinvent-3.7-biosphere,sqlite,4318,biosphere
ecoinvent-3.7-cutoff,sqlite,19128,background
MyForeground,sqlite,0,foreground


# Introduction to Numpy

Numpy is a python libray for symbolic calculus. 

You write Sympy expression as you write **standard python expressions**, using **sympy symbols** in them. 

The result is then a **symbolic expression that can be manipulated**, instead of a **numeric value**.

In [5]:
from sympy import symbols 

# create sympy symbol
x = symbols("x")

# Expressions are not directly evaluated 
f = x * 2 + 4 
f

In [6]:
# symbols can be replaced by values afterwards 
f.subs(dict(x=3))

In practice, you don't need to care about Sympy. Just remember that : 
* The parameters defined below are **instances of sympy symbols**
* Any **valid python expression** containing a **sympy symbol** will create a **sympy symbolic expression**

# Define input parameters

First, we define the input parameters of the model together with their distribution.

The numeric parameters are **instances of sympy 'Symbol'**. 

Thus, any python arithmetic expression composed of parameters will result in a **symbolic expression** to be used later in the definition of the model, rather than a static numeric result.

In [7]:
# Example of 'float' parameters
a = agb.newFloatParam(
    'a', 
    default=0.5, min=0.2, max=2,  distrib=DistributionType.TRIANGLE, # Distribution type, linear by default
    description="hello world",
    label="extended label for a")

b = agb.newFloatParam(
    'b',
    default=0.5, # Fixed if no min /max provided
    distrib=DistributionType.FIXED,
    description="foo bar")

share_recycled_aluminium = agb.newFloatParam(
    'share_recycled_aluminium',  
    default=0.6, min=0, max=1, std=0.2, distrib=DistributionType.NORMAL, # Normal distrib, with std dev
    description="Share of reycled aluminium")

c = agb.newFloatParam(
    'c',  
    default=0.6, std=0.2, distrib=DistributionType.LOGNORMAL)

beta = agb.newFloatParam(
    'beta',  
    default=0.6, std=0.2, a=2, b=5, distrib=DistributionType.BETA)

# You can define boolean parameters, taking only discrete values 0 or 1
bool_param = agb.newBoolParam(
    'bool_param', 
    default=1)

# Example 'enum' parameter, acting like a switch between several possibilities
# Enum parameters are not Symbol themselves
# They are a facility to represent many boolean parameters at once '<paramName>_<enumValue>' 
# and should be used with the 'newSwitchAct' method 
elec_switch_param = agb.newEnumParam(
    'elec_switch_param', 
    values=["us", "eu"], # If provided as list, all possibilities have te same probability
    default="us", 
    description="Switch on electricty mix")

# Another example enum param
techno_param = agb.newEnumParam(
    'techno_param', 
    values={
        "technoA":0.4, 
        "technoB":0.1,
        "technoC":0.5}, # You can provide a statistical weight for each value, by using a dict
    default="technoA", 
    description="Choice of techonoly")

## Persistance of parameters

By default, new parameters are kept in memory but also persisted in the project (unless save=False).
You can persist parameters afterwards with `persistParams()`.
You can load also load parameters from an existing database with `loadParams()`.
The persistance of parameters and the distribution is compatible with **Brightway2** and **Activity Browser**  [see documentation of stat_arrays](https://stats-arrays.readthedocs.io/en/latest/)

In [8]:
# Load parameters previously  persisted in the dabatase.
agb.loadParams()

[ParamRegistry] Param a was already defined in '<project>' : overriding.
[ParamRegistry] Param a was already defined in '<project>' : overriding.
[ParamRegistry] Param b was already defined in '<project>' : overriding.
[ParamRegistry] Param b was already defined in '<project>' : overriding.
[ParamRegistry] Param share_recycled_aluminium was already defined in '<project>' : overriding.
[ParamRegistry] Param share_recycled_aluminium was already defined in '<project>' : overriding.
[ParamRegistry] Param c was already defined in '<project>' : overriding.
[ParamRegistry] Param c was already defined in '<project>' : overriding.
[ParamRegistry] Param beta was already defined in '<project>' : overriding.
[ParamRegistry] Param beta was already defined in '<project>' : overriding.
[ParamRegistry] Param bool_param was already defined in '<project>' : overriding.
[ParamRegistry] Param bool_param was already defined in '<project>' : overriding.
[ParamRegistry] Param elec_switch_param was already de

# Manage several Datasets

lca_algebraic supports several foreground / background datasets. Background datasets are considered static / non parametrized by the library : they use standard LCA method of **Brightway2**. 

Foreground databases are considered parametric and their activities are developped as functions of parameters and background activities.

## Set status of a database

The functions **setForeground(...)** and **setBackground(...)** change the status of a database.

In [9]:
agb.setForeground(USER_DB)
agb.list_databases()

,backend,nb_activities,type
name,,,
ecoinvent-3.7-biosphere,sqlite,4318,biosphere
ecoinvent-3.7-cutoff,sqlite,19128,background
MyForeground,sqlite,0,foreground


## Import / export

`lca_algebraic` extends [BW2Package](https://2.docs.brightway.dev/technical/bw2io.html), adding persistence of parameters.

In [10]:
# Save database and parameters as Bzipped JSON
agb.export_db(USER_DB, "db.bw2")

In [11]:
# Reimport DB
agb.import_db("db.bw2")

[ParamRegistry] Param a was already defined in '<project>' : overriding.
[ParamRegistry] Param a was already defined in '<project>' : overriding.
Variable 'a' was already defined : overidding it with param.
[ParamRegistry] Param b was already defined in '<project>' : overriding.
[ParamRegistry] Param b was already defined in '<project>' : overriding.
Variable 'b' was already defined : overidding it with param.
[ParamRegistry] Param share_recycled_aluminium was already defined in '<project>' : overriding.
[ParamRegistry] Param share_recycled_aluminium was already defined in '<project>' : overriding.
Variable 'share_recycled_aluminium' was already defined : overidding it with param.
[ParamRegistry] Param c was already defined in '<project>' : overriding.
[ParamRegistry] Param c was already defined in '<project>' : overriding.
Variable 'c' was already defined : overidding it with param.
[ParamRegistry] Param beta was already defined in '<project>' : overriding.
[ParamRegistry] Param beta 

Brightway2 SQLiteBackend: MyForeground

## Freeze 

A foreground database can be "frozen" to be used as a background database for a specific scenario : the parametrized amounts in the exhanges are computed for a given configuration of the parameters, and replaced by their value. The formulas are still stored in the database and not lost : the database can still be used as a foreground database until its status is changed with `setBackground(...)`.

This feature is useful for studies requiring several datasets to be used as background by other ones. It also enables to use standard Brightway2 tools, not aware of parametrization. 


In [12]:
agb.freezeParams(
    USER_DB, # Name of database to freeze
    
    a=1, b=2) # custom parameter values

# Get references to background activities

We provide two functions for easy and fast (indexed) search of activities in reference databases : 
* **findBioAct** : Search activity in **biosphere3** db
* **findTechAct** : Search activity in **ecoinvent** db

Those methods are **faster** and **safer** than using traditionnal "list-comprehension" search : 
They will **fail with an error** if **more than one activity** matches, preventing the model to be based on a random selection of one activity.


In [13]:
# Biosphere activities
ground_occupuation = agb.findBioAct('Occupation, industrial area') # Search by name
heat = agb.findBioAct('Heat, waste', categories=['air']) # Add category selector

# Technosphere activities

# You can add an optionnal location selector
alu = agb.findTechAct("aluminium alloy production, AlMg3", loc="RER")
alu_scrap = agb.findTechAct('aluminium scrap, new, Recycled Content cut-off')

# Elec 
eu_elec = agb.findTechAct("market group for electricity, medium voltage", 'ENTSO-E')
us_elec = agb.findTechAct("market group for electricity, medium voltage", 'US')

chromium = agb.findTechAct("market for chromium oxide, flakes")

# Define the model

The model is defined as a nested combination of background activities with amounts.

Amounts are defined either as constant float values or algebric formulas implying the parameters defined above.

## Create new activities

In [14]:
# Create a new activity
activity1 = agb.newActivity(USER_DB, # We define foreground activities in our own DB
    "first foreground activity", # Name of the activity
    "kg", # Unit
    exchanges= { # We define exhanges as a dictionarry of 'activity : amount'
        ground_occupuation:3 * b, # Amount can be a fixed value 
        heat: b + 0.2  # Amount can be a Sympy expression (any arithmetic expression of Parameters)
    })

# You can create a virtual "switch" activity combining several activities with an Enum parameter
elec_mix = agb.newSwitchAct(USER_DB, 
    "elect mix", # Name
    elec_switch_param, # Sith parameter
    { # Dictionnary of enum values / activities
        "us" : us_elec, # By default associated amount is 1
        "eu" : (eu_elec, 0.8)  # You can also provide custom amout or formula with a tuple 
    })

## Copy and update existing activity

You can copy and update an existing background activity.

Several new helper methods have been added to the class **Activity** for easy update of exchanges.

In [15]:
alu2 = agb.copyActivity(
    USER_DB, # The copy of a background activity is done in our own DB, so that we can safely update it                
    alu, # Initial activity : won't be altered
    "Aluminium 2") # New name

# Update exchanges by their name 
alu2.updateExchanges({
    
    # Update amount : the special symbol *old_amount* references the previous amount of this exchange
    "aluminium, cast alloy": agb.old_amount * (1 - share_recycled_aluminium),
    
    # Update input activity. Note also that you can use '*' wildcard in exchange name
    "electricity*": elec_mix,
    
    # Update both input activity and amount. 
    # Note that you can use '#' for specifying the location of exchange (useful for duplicate exchange names)
    "chromium#GLO" : dict(amount=4.0, input=chromium)
}) 

# Add exchanges 
alu2.addExchanges({alu_scrap :  12})

## Final model


In [35]:
total_inventory = agb.newActivity(USER_DB, "total_inventory", "kg", {
    activity1 : b * 5 + a + 1, # Reference the activity we just created
    alu2: 3 * share_recycled_aluminium, 
    alu:0.4 * a})

Activity 'total_inventory' was already in 'MyForeground'. Overwriting it


## Or load existing model /activities from database

Alternatively, you may not define the model again, but load it from the USER DB.

In [17]:
activity1 = agb.findActivity("first foreground activity", db_name=USER_DB)
total_inventory = agb.findActivity("total_inventory", db_name=USER_DB)
alu2 = agb.findActivity("Aluminium 2", db_name=USER_DB)

## Display activities

**printAct** displays the list of all exchanges of an activity.

Note that symbolic expressions have not been evaluated at this stage

In [18]:
# Print_act displays activities as tables
agb.printAct(activity1) 

first foreground activity (1.000000 kg)           \
                                                              input   amount   
Heat, waste                                             Heat, waste  b + 0.2   
Occupation, industrial area             Occupation, industrial area      3*b   

                                                
                                          unit  
Heat, waste                          megajoule  
Occupation, industrial area  square meter-year

In [19]:
agb.printAct(total_inventory)

total_inventory (1.000000 kg)  \
                                                                    input   
Aluminium 2                                          Aluminium 2[RER]{FG}   
aluminium alloy production, AlMg3  aluminium alloy production, AlMg3[RER]   
first foreground activity                   first foreground activity{FG}   

                                                                         
                                                       amount      unit  
Aluminium 2                        3*share_recycled_aluminium  kilogram  
aluminium alloy production, AlMg3                       0.4*a  kilogram  
first foreground activity                         a + 5*b + 1        kg

In [20]:
# You can also compute amounts by replacing parameters with a float value 
agb.printAct(activity1, b=1.5)

first foreground activity (1.000000 kg)  \
                                                              input   
Heat, waste                                             Heat, waste   
Occupation, industrial area             Occupation, industrial area   

                                                                  
                                       amount               unit  
Heat, waste                  1.70000000000000          megajoule  
Occupation, industrial area  4.50000000000000  square meter-year

In [21]:
# You can print several activities at once to compare them
agb.printAct(alu, alu2)

# Select the impacts to consider

In [24]:
# List of impacts to consider
impacts = agb.findMethods("climate change", mainCat="EF v3.0")
impacts

[('EF v3.0', 'climate change', 'global warming potential (GWP100)'),
 ('EF v3.0', 'climate change: biogenic', 'global warming potential (GWP100)'),
 ('EF v3.0', 'climate change: fossil', 'global warming potential (GWP100)'),
 ('EF v3.0',
  'climate change: land use and land use change',
  'global warming potential (GWP100)')]

# Impacts

## Define functional unit

The functional unit is a quantity that can be parametrized


In [25]:
functional_value = a + 5

## Compute impacts

In [52]:
agb.compute_impacts(
    
    # Root activity of our inventory
    total_inventory, 
    
    # list of impacts to consider
    impacts, 
    
    # The impaxts will be divided by the functional unit
    functional_unit=functional_value,
    
    # Parameters of the model
    a=1.0,
    elec_switch_param="us",
    share_recycled_aluminium=0.4)

[INFO] Db changed recently, clearing cache expr
[INFO] Db changed recently, clearing cache lcia
Error while parsing formula 'CORINE 122c' : backing to amount
Error while parsing formula 'CORINE x' : backing to amount
Error while parsing formula 'CORINE 122b' : backing to amount
Error while parsing formula 'CORINE x' : backing to amount
Error while parsing formula 'CORINE 121a' : backing to amount
Error while parsing formula 'CORINE 122c' : backing to amount
Error while parsing formula 'CORINE 122b' : backing to amount
Error while parsing formula 'CORINE x' : backing to amount
Error while parsing formula 'CORINE 31' : backing to amount
Error while parsing formula 'CORINE 121a' : backing to amount
Error while parsing formula 'CORINE 21' : backing to amount
Error while parsing formula 'CORINE 122a' : backing to amount
Error while parsing formula 'CORINE 231' : backing to amount
Error while parsing formula 'CORINE 231' : backing to amount
Error while parsing formula 'CORINE 122a' : backing

,climate change - global warming potential (GWP100)[kg CO2-Eq],climate change: biogenic - global warming potential (GWP100)[kg CO2-Eq],climate change: fossil - global warming potential (GWP100)[kg CO2-Eq],climate change: land use and land use change - global warming potential (GWP100)[kg CO2-Eq]
total_inventory,0,0,0,0


In [ ]:
# We can provide list of values as parameters


# Compute with algebric implementation
# The values should be the same as above
agb.compute_impacts(
    total_inventory, # The model 
    impacts, # Impacts
    functional_unit = functional_value, 
    
    # Parameters of the model
    a=[1, 2, 3], 
    elec_switch_param=["us", 'us', 'us'],
    share_recycled_aluminium=0.4)

In [51]:
# You can compute several LCAs at a time and compare them:
agb.compute_impacts(
    [alu, alu2], # The models
    
    impacts, # Impacts
    
    # Parameters of the model
    share_recycled_aluminium=0.3,
    elec_switch_param="us")

,climate change - global warming potential (GWP100)[kg CO2-Eq],climate change: biogenic - global warming potential (GWP100)[kg CO2-Eq],climate change: fossil - global warming potential (GWP100)[kg CO2-Eq],climate change: land use and land use change - global warming potential (GWP100)[kg CO2-Eq]
"aluminium alloy production, AlMg3[RER]",7.30938,0.0158928,7.2762,0.0172852
Aluminium 2[RER],30.5997,0.0676298,30.5064,0.0256945


## Fast computation of many parameter values

In [52]:
# Fast computation for millions of separate samples
agb.compute_impacts(
    total_inventory, # The model 
    impacts, # Impacts
    functional_unit = functional_value,
    
    # Parameters of the model
    a=list(range(1, 100000)), # All lists should have the same size
    share_recycled_aluminium=1, # Those parameters are fixed
    elec_switch_param="eu")

,climate change - global warming potential (GWP100)[kg CO2-Eq],climate change: biogenic - global warming potential (GWP100)[kg CO2-Eq],climate change: fossil - global warming potential (GWP100)[kg CO2-Eq],climate change: land use and land use change - global warming potential (GWP100)[kg CO2-Eq]
a,,,,
1,13.6403,0.0323597,13.5989,0.00906543
2,12.1094,0.028645,12.072,0.00875809
3,10.9612,0.025859,10.9268,0.00852759
4,10.0681,0.0236922,10.0361,0.00834831
5,9.35367,0.0219587,9.32351,0.00820489
...,...,...,...,...
99995,2.92439,0.00635869,2.91112,0.00691421
99996,2.92439,0.00635869,2.91112,0.00691421
99997,2.92439,0.00635869,2.91112,0.00691421


## Split impacts along axis

It is possible to **tag** activities and then ventilate the impacts according to the value of this "tag".
This is useful to split impact by *phase* or *sub module*.

In [53]:
# Tag activities with a custom attribute : 'phase' in this case
alu2.updateMeta(phase= "phase a")
activity1.updateMeta(phase= "phase b")

In [64]:
# Provide the name of the custom attribute as 'axis'
# The impacts are split between those
agb.compute_impacts(
    total_inventory, # The model
    impacts, # Impacts
    
    functional_unit = functional_value,
    axis="phase",

    
    # Parameters
    a=1.0,
    elec_switch_param="us",
    share_recycled_aluminium=0.4)

KeyboardInterrupt: 

 # Sensitivity analysis 
 
 ## One at a time 
 
 We provide several functions for computing **statistics** for **local variations** of parameters (one at a time).
 
 ### oat_matrix(model, impacts)
 
 Shows a **matrix of impacts x parameters** colored according to the variation of the impact in the bounds of the parameter.


In [ ]:
oat_matrix(
    total_inventory, 
    impacts, 
    functional_unit=functional_value)

### oat_dashboard_matrix

This functions draws a dashboard showing :
* A dropdown list, for choosing a parameter
* Several graphs of evolution of impacts for this parameter
* Full table of data
* A graph of "bars" representing the variation of each impact for this parameter (similar to the information given in oat_matrix) 

In [ ]:
oat_dashboard_interact(
    total_inventory, 
    impacts, 
    functional_unit=functional_value,
    
    # Optionnal layout parameters
    figspace=(0.5,0.5),
    figsize=(15, 15),
    sharex=True)

## Monte-carlo methods & Sobol indices

Here we leverage fast computation of monte-carlo approches. 

We compute **global sensivity analysis** (GSA).
Not only local ones.


### Sobol Matrix 

Similar to OAT matrix, we compute Sobol indices. they represent the ratio between the variance due to a given parameter and the total variance.

for easier comparison, we translate those relative sobol indices into "deviation / mean" importance :

$$RelativeDeviation = \frac{\sqrt{sobol(param) \times totalVariance(impact))}}{mean(impact)}$$




In [ ]:
# Show sobol indices 
incer_stochastic_matrix(
    total_inventory, 
    impacts, 
    functional_unit=functional_value)

###  Graphs of impacts and their distribution

We provide a dashboard showing **violin graphs** : the exact probabilistic distribution for each impact. Together with medians of the impacts.

In [ ]:
incer_stochastic_violin(
    total_inventory, impacts,
    functional_unit=functional_value,
    
    # Optionnal layout parameters
    figspace=(0.5,0.5),
    figsize=(15, 15),
    sharex=True, 
    nb_cols=3)

In [ ]:
##### Alternatively, graphs can be shown horizontally, together with a box of statistical outcomes
distrib(
    total_inventory, impacts,
    functional_unit=functional_value,
    
    # Optionnal layout parameters
    height=7, width=15,
    nb_cols=2,
    percentiles=[5, 95])

### Full dashboard

A dashboard groups all this information in a single interface with tabs.

It also shows total variation of impacts. This last graph could be improved by showing stacked colored bars with the contribution of each parameter to this variation, according to Sobol indices. 

In [ ]:
incer_stochastic_dashboard(
    model=total_inventory, 
    methods=impacts,
    functional_unit=functional_value)

# Producing simplified models 

One of te outcome of the statisticall analysis above would be to identify main input parameters and produce simplidied models, fixing the minor ones.

We provide several functions for doing this.

## Explore initial algrebraic model

In [ ]:
# First, let's look at the full expression defining our model
expr, _ = actToExpression(total_inventory)
expr

## Compute simplified models

We provide some method to automatically select a subset of parameters, based on the **sobol indices**, and then compute simplified models for it.

We also round numerical expression to 3 digits, and we remove terms in sums that are less than 1% of total.

In [ ]:
simplified = sobol_simplify_model(
    total_inventory, # The model
    impacts, # Impacts to consider
    functional_unit=functional_value,
    
    n=10000, # For large model, you may test other value and ensure ST and sum(S1) are close to 1.0 
    fixed_mode = FixedParamMode.MEDIAN, # We replace minor parameters by median by default,
    min_ratio=0.8, # Min ratio of variability to explain

    num_digits=3)

In [ ]:
# Let's look at the expression for first impact again 
# much simpler ! 
simplified[0].expr

## Compare simplified model with full model

Finally, we can compare the distribution of those simplified model against the full model. We provide a function for graphical display of it, and compuation of de R-Square score.


In [ ]:
compare_simplified(
    total_inventory, 
    impacts, 
    simplified,
    functional_unit=functional_value)